In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("C:\\Users\\Lenovo\\Downloads\\Infosys 6.0\\archive (1)\\customer_support_tickets.csv")

In [3]:
df.head()

,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8469 entries, 0 to 8468
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Ticket ID                     8469 non-null   int64  
 1   Customer Name                 8469 non-null   object 
 2   Customer Email                8469 non-null   object 
 3   Customer Age                  8469 non-null   int64  
 4   Customer Gender               8469 non-null   object 
 5   Product Purchased             8469 non-null   object 
 6   Date of Purchase              8469 non-null   object 
 7   Ticket Type                   8469 non-null   object 
 8   Ticket Subject                8469 non-null   object 
 9   Ticket Description            8469 non-null   object 
 10  Ticket Status                 8469 non-null   object 
 11  Resolution                    2769 non-null   object 
 12  Ticket Priority               8469 non-null   object 
 13  Tic

Data cleaning

In [5]:
df.isnull().sum()

Ticket ID                          0
Customer Name                      0
Customer Email                     0
Customer Age                       0
Customer Gender                    0
Product Purchased                  0
Date of Purchase                   0
Ticket Type                        0
Ticket Subject                     0
Ticket Description                 0
Ticket Status                      0
Resolution                      5700
Ticket Priority                    0
Ticket Channel                     0
First Response Time             2819
Time to Resolution              5700
Customer Satisfaction Rating    5700
dtype: int64

In [6]:
df.shape

(8469, 17)

In [7]:
#Handle Missing Values
# Resolution only exists for closed tickets
df['Resolution'] = df['Resolution'].fillna("Not Resolved Yet")

# Satisfaction rating not applicable for open tickets
df.loc[df['Ticket Status'] != 'Closed', 'Customer Satisfaction Rating'] = np.nan

In [ ]:
df = df[(df['Customer Age'] > 0) & (df['Customer Age'] <= 100)]
#Removes invalid age values

In [ ]:
#Date Conversion
date_cols = [
    'Date of Purchase',
    'First Response Time',
    'Time to Resolution'
]

for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')
#Converts string-based dates into datetime objects

In [ ]:
#Feature Engineering
df['Resolution Time (Hours)'] = (
    (df['Time to Resolution'] - df['First Response Time'])
    .dt.total_seconds() / 3600
)
#Creates a meaningful operational metric

In [ ]:
#Text cleaning 
def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()
#Converts unstructured text into a normalized form.Removes punctuation, noise, and inconsistent formatting,

In [ ]:
import re
#Text Cleaning
df['Ticket Subject'] = df['Ticket Subject'].apply(clean_text)
df['Ticket Description'] = df['Ticket Description'].apply(clean_text)
#Ensures consistent text across all records

In [ ]:
#Deduplication
before = df.shape[0]
df.drop_duplicates(inplace=True)
after = df.shape[0]

print(f"Duplicates removed: {before - after}")
#Prevents double counting of tickets

Duplicates removed: 0


In [ ]:
#Standardization
df['Ticket Status'] = df['Ticket Status'].str.upper()
df['Ticket Priority'] = df['Ticket Priority'].str.upper()
df['Ticket Channel'] = df['Ticket Channel'].str.lower()
#Removes inconsistencies like open, Open, OPEN

In [17]:
#Knowledge Graph IDs
df['Customer_ID'] = df['Customer Email'].astype(str)
df['Product_ID'] = df['Product Purchased'].astype(str)
df['Ticket_ID'] = df['Ticket ID'].astype(str)


Data enrinchment

In [18]:
# 1. Customer Segment (derived knowledge)
def customer_segment(age):
    if age < 25:
        return "YOUTH"
    elif age < 45:
        return "ADULT"
    elif age < 60:
        return "MIDDLE_AGED"
    else:
        return "SENIOR"

df['Customer_Segment'] = df['Customer Age'].apply(customer_segment)

In [19]:
# 2. SLA Breach Flag (business enrichment)
df['SLA_Breached'] = df['Resolution Time (Hours)'].apply(
    lambda x: 1 if pd.notna(x) and x > 24 else 0
)

In [ ]:
# 3. Ticket Urgency Score (rule-based AI)
priority_map = {
    'LOW': 1,
    'MEDIUM': 2,
    'HIGH': 3,
    'CRITICAL': 4
}

df['Urgency_Score'] = df['Ticket Priority'].map(priority_map)

#Converts categorical ticket priority into a numeric severity score

ENCODING + NORMALIZATION

In [21]:
df_ml = df.copy()


In [22]:
df_ml['Resolution Time (Hours)'] = (
    (df_ml['Time to Resolution'] - df_ml['First Response Time'])
    .dt.total_seconds() / 3600
)


In [23]:
#numerical columns
from sklearn.preprocessing import MinMaxScaler

num_cols = [
    'Customer Age',
    'Resolution Time (Hours)',
    'Customer Satisfaction Rating'
]

scaler = MinMaxScaler()
df_ml[num_cols] = scaler.fit_transform(df_ml[num_cols])


In [24]:
#encoding
from sklearn.preprocessing import LabelEncoder

cat_cols = [
    'Ticket Status',
    'Ticket Priority',
    'Ticket Channel',
    'Ticket Type'
]

label_encoders = {}

for col in cat_cols:
    le = LabelEncoder()
    df_ml[col + '_ENCODED'] = le.fit_transform(df_ml[col])
    label_encoders[col] = le


In [25]:
df_ml.head()


,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,...,Customer_ID,Product_ID,Ticket_ID,Customer_Segment,SLA_Breached,Urgency_Score,Ticket Status_ENCODED,Ticket Priority_ENCODED,Ticket Channel_ENCODED,Ticket Type_ENCODED
0,1,Marisa Obrien,carrollallison@example.com,0.269231,Other,GoPro Hero,2021-03-22,Technical issue,product setup,i m having an issue with the product purchased...,...,carrollallison@example.com,GoPro Hero,1,ADULT,0,4,2,0,3,4
1,2,Jessica Rios,clarkeashley@example.com,0.461538,Female,LG Smart TV,2021-05-22,Technical issue,peripheral compatibility,i m having an issue with the product purchased...,...,clarkeashley@example.com,LG Smart TV,2,ADULT,0,4,2,0,0,4
2,3,Christopher Robbins,gonzalestracy@example.com,0.576923,Other,Dell XPS,2020-07-14,Technical issue,network problem,i m facing a problem with my product purchased...,...,gonzalestracy@example.com,Dell XPS,3,MIDDLE_AGED,0,1,0,2,3,4
3,4,Christina Dillon,bradleyolson@example.org,0.173077,Female,Microsoft Office,2020-11-13,Billing inquiry,account access,i m having an issue with the product purchased...,...,bradleyolson@example.org,Microsoft Office,4,ADULT,0,1,0,2,3,0
4,5,Alexander Carroll,bradleymark@example.com,0.942308,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,data loss,i m having an issue with the product purchased...,...,bradleymark@example.com,Autodesk AutoCAD,5,SENIOR,0,1,0,2,1,0
